# Hermitage I5 migration notebook

This notebook app will run the ETL process to migrate Heremitage's I5 to a new Odoo 15 system.
It will read csv/tsv files from the input_csv_files, make the necessary transformations to make it importable in Odoo 15 and then load the data it into Hermitage's new Odoo 15 instance.

### Imports
- pandas: to make transformations on the data
- Models' migration config
- Import function

In [ ]:
import re
import pandas

from models_migration_config import models_migration_config

from import_functions import import_data, import_ignored_fields

INPUT_CSV_FILES_PATH = 'input_csv_files/'
GENERATED_CSV_FILES_PATH = 'generated_csv_files/'



**Transform Vendors from I5**

In [ ]:
vendors_file_path = f'{INPUT_CSV_FILES_PATH}APMASTER - Vendors.tsv'
vendors_dataframe = pandas.read_csv(vendors_file_path, sep='\t')
vendors_dataframe = vendors_dataframe.fillna('')
# AMVEND	AMNAME	AMADR1	AMADR2	AMADR3	AMTELE	AMFRGT	actual_term
# 1	BROAN MFG OR EDN	4641 PAYSPHERE CIRCLE	CHICAGO,IL 60674         		8778470145	1250	2% 10TH PROX
# 172	HUNTER FAN COMPANY	P O BOX 19773		PALATINE IL 60055-9773	9017441200	1000	1% 20TH
# 10003	LEDVANCE OR IMARK	P O BOX 72524	(OSRAM)	CLEVELAND OH 44192	8002555042	1000	2% 90Days
# 10004	ADVANCE TRANSFORMER CO	P O BOX 100332	ATLANTA GA  30384		0	750	2% 10TH PROX
# 10005	LUTRON ELECTRONICS CO INC	P O BOX 644396		PITTSBURGH, PA 15264-4396	8005239466	500	1%10TH PROX

col_names = ['AMVEND', 'AMNAME', 'AMADR1', 'AMADR2', 'AMADR3', 'AMTELE', 'AMFRGT', 'actual_term']
for col_name in col_names:
    vendors_dataframe[col_name] = vendors_dataframe[col_name].astype(str)
    vendors_dataframe[col_name] = vendors_dataframe[col_name].str.strip()

# Move AMADR1 column to the end for better visibility
vendors_dataframe = vendors_dataframe[['AMVEND', 'AMNAME', 'AMADR2', 'AMADR3', 'AMTELE', 'AMFRGT', 'actual_term', 'AMADR1']]
vendors_dataframe.rename(columns={'AMADR1': 'street'}, inplace=True)

vendors_dataframe['street2'] = ''
vendors_dataframe['city'] = ''
vendors_dataframe['state_code'] = ''
vendors_dataframe['zip'] = ''

def is_zip(zip_code : str) -> bool:
    """
    >>> is_zip('6067')
    True
    >>> is_zip('60674')
    True
    >>> is_zip('60674-1234')
    True
    """
    return bool(re.match(r'^\d{5}(-\d{4})?$', zip_code)) or bool(re.match(r'^\d{4}', zip_code))

us_states = {
    'AL', 'AK', 'AZ', 'AR', 'CA', 'CO', 'CT', 'DE', 'FL', 'GA', 'HI', 'ID', 'IL', 'IN', 'IA', 'KS', 'KY',
    'LA', 'ME', 'MD', 'MA', 'MI', 'MN', 'MS', 'MO', 'MT', 'NE', 'NV', 'NH', 'NJ', 'NM', 'NY',
    'NC', 'ND', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX', 'UT', 'VT', 'VA', 'WA', 'WV', 'WI', 'WY'
}

def is_a_state(state : str) -> bool:
    return state in us_states

def clean_address_field(address_field: str) -> str:
    address_field = address_field.strip()
    address_field = re.sub(r'\s+', ' ', address_field)
    address_field = address_field.replace(',', ' ')
    return address_field


for row_index, row in vendors_dataframe.iterrows():
    row_address2 = row.AMADR2 or ''
    row_address2 = clean_address_field(row_address2)
    row_address3 = row.AMADR3 or ''
    row_address3 = clean_address_field(row_address3)

    if row_address3:
        row.street2 = row_address2
        city_state_zip = row_address3
    else:
        city_state_zip = row_address2
    if city_state_zip:
        city_state_zip_splitted = city_state_zip.split(' ')[::-1]

        for i, address_field in enumerate(city_state_zip_splitted):
            if is_zip(address_field):
                row.zip = address_field.strip()
            elif is_a_state(address_field):
                row.state_code = address_field.strip()
            else:
                row.city = ' '.join(city_state_zip_splitted[i:][::-1]).strip()
                break
        if not row.zip or not row.state_code or not row.city and row.street2:
            city_state_zip_splitted = row.street2.split(' ')[::-1]
            for i, address_field in enumerate(city_state_zip_splitted):
                if not row.zip and is_zip(address_field):
                    row.zip = address_field.strip()
                elif not row.state_code and is_a_state(address_field):
                    row.state_code = address_field.strip()
                elif not row.city:
                    row.city = ' '.join(city_state_zip_splitted[i:][::-1]).strip()
                    break

states_dataframe = pandas.read_csv(f'{INPUT_CSV_FILES_PATH}state_codes.csv')
# Merge with states to get the state and country external id
vendors_dataframe = pandas.merge(vendors_dataframe, states_dataframe, on='state_code', how='left')

# Merge with the payment terms to get the payment term external id
payment_terms_dataframe = pandas.read_csv(f'{INPUT_CSV_FILES_PATH}account.payment.term.csv')
payment_terms_dataframe.rename(columns={'name': 'actual_term', 'id': 'property_supplier_payment_term_id/id'}, inplace=True)
merge_payment_terms_dataframe = payment_terms_dataframe[['actual_term', 'property_supplier_payment_term_id/id']]

vendors_dataframe = pandas.merge(vendors_dataframe, merge_payment_terms_dataframe, on='actual_term', how='left')
# remove unneeded columns
vendors_dataframe.drop(columns=['AMADR2', 'AMADR3', 'AMFRGT', 'actual_term', 'state_code'], inplace=True)
# rename columns to match the odoo model
vendors_dataframe.rename(columns={
    'AMVEND': 'vendor_account_number',
    'AMNAME': 'name',
    'AMTELE': 'phone'
}, inplace=True)

vendors_dataframe.insert(0, 'supplier_rank', 1)
# add Vendor tag
vendors_dataframe.insert(0, 'category_id/id', '__export__.res_partner_category_56_7fafda33')
# add id column populated with ids generated from 'i5_migration.res.partner.vendors_{row_index}'
vendors_dataframe.insert(0, 'id', [f'i5_migration.res.partner.vendor_{row_index}' for row_index in range(1, 1 + len(vendors_dataframe))])

vendors_dataframe.to_csv(f'{GENERATED_CSV_FILES_PATH}i5.vendors.csv', index=False, sep=',')


**Load vendors (res.partner)**

In [ ]:
import_data(file_csv='i5.vendors.csv', model_name='res.partner')

**Transform locations from I5**

In [ ]:

model_name = 'stock.location'

company_id = 'base.main_company'

locations_dataframe = pandas.read_csv(f'{INPUT_CSV_FILES_PATH}RFLOCATE.tsv', sep='\t')
# locations_dataframe = locations_dataframe.fillna('')
# #cast all columns to str
locations_dataframe = locations_dataframe.astype(str)
# #Trimm spaces
locations_dataframe = locations_dataframe.applymap(lambda x: x.strip() if isinstance(x, str) else x)


#Format of i5 locations file
# id  name
# 9010101	INVENTORY LOCATION  
# 9060101	                    
# 9200101	COMM WHSE           
# 9270101	                    
# 9270201	                    
# 9270301	                    
# 9270401	                    
# 9270501	                    
# 9270601	                    
# 9270701	                    

# Main stock.location id: stock.stock_location_stock

# Columns needed for Odoo's locations dataframe
# id,name,usage,company_id/id,parent_id/id,temporary_parent_id

# The locations hierarchy is as follows:
# 1 - stock.stock_location_stock --> i5 Value: 9
# 2 - Aisle
# 3 - Section(venv) fernando@fernando-Z370-AORUS-Ultra-Gaming:/media/fernando/Shared/hermitage$ 
# 4 - INVENTORY LOCATION (if it has a name the shelf location is called by the name, otherwise it is called by the id)
# Would generate the following locations:
# i5_migration.stock.location_aisle_01,Aisle 01,internal,base.main_company,stock.stock_location_stock,stock.stock_location_stock
# i5_migration.stock.location_section_01_01,Section 01,internal,base.main_company,i5_migration.stock.location_aisle_01,stock.stock_location_stock
# i5_migration.stock.location_shelf_01_01_01,Shelf 01 (INVENTORY LOCATION),internal,base.main_company,i5_migration.stock.location_section_01_01,stock.stock_location_stock

# Example 2: for location 9060101
# 1 - stock.stock_location_stock (already created)
# 2 - Aisle 06
# 3 - Section 01
# 4 - Shelf 01
# Would generate the following locations:
# i5_migration.stock.location_aisle_06,Aisle 06,internal,base.main_company,stock.stock_location_stock,stock.stock_location_stock
# i5_migration.stock.location_section_06_01,Section 01,internal,base.main_company,i5_migration.stock.location_aisle_06,stock.stock_location_stock
# i5_migration.stock.location_shelf_06_01_01,Shelf 01,internal,base.main_company,i5_migration.stock.location_section_06_01,stock.stock_location_stock


transformation_result_dataframe = pandas.DataFrame(columns=['id', 'name', 'usage', 'company_id/id', 'location_id/id', 'temporary_parent_id'])
aisles_ids = set()
section_ids = set()
shelf_ids = set()
for location in locations_dataframe.iterrows():
    ## Aisle ##
    aisle_number = str(location[1].id)[1:3]
    aisle_id = f'i5_migration.stock.location_aisle_{aisle_number}'
    if aisle_id not in aisles_ids:
        aisles_ids.add(aisle_id)
        transformation_result_dataframe = transformation_result_dataframe.append({
            'id': f'i5_migration.stock.location_aisle_{aisle_number}',
            'name': f'Aisle {aisle_number}',
            'usage': 'internal',
            'company_id/id': 'base.main_company',
            'location_id/id': 'stock.stock_location_stock',
            'temporary_parent_id': 'stock.stock_location_stock'
        }, ignore_index=True)

    ## Section ##
    section_number = str(location[1].id)[3:5]
    section_id = f'i5_migration.stock.location_section_{aisle_number}_{section_number}'
    if section_id not in section_ids:
        section_ids.add(section_id)
        transformation_result_dataframe = transformation_result_dataframe.append({
            'id': f'i5_migration.stock.location_section_{aisle_number}_{section_number}',
            'name': f'Section {section_number}',
            'usage': 'internal',
            'company_id/id': 'base.main_company',
            'location_id/id': f'i5_migration.stock.location_aisle_{aisle_number}',
            'temporary_parent_id': 'stock.stock_location_stock'
        }, ignore_index=True)

    ## Shelf ##
    shelf_number = str(location[1].id)[5:7]
    shelf_id = f'i5_migration.stock.location_shelf_{aisle_number}_{section_number}_{shelf_number}'
    loc_name = str(location[1].description).strip()
    if shelf_id not in shelf_ids:
        shelf_ids.add(shelf_id)
        if not loc_name:
            shelf_name = f'Shelf {shelf_number}'
        else:
            shelf_name = f'Shelf {shelf_number} ({loc_name})'
        transformation_result_dataframe = transformation_result_dataframe.append({
            'id': shelf_id,
            'name': shelf_name,
            'usage': 'internal',
            'company_id/id': 'base.main_company',
            'location_id/id': f'i5_migration.stock.location_section_{aisle_number}_{section_number}',
            'temporary_parent_id': 'stock.stock_location_stock',
            'location_number': location[1].id
        }, ignore_index=True)

# Create import file with the locations with temporary parent
odoo_locations_temp_parent_dataframe = transformation_result_dataframe.copy(deep=True)
odoo_locations_temp_parent_dataframe.drop(columns=['location_id/id'], inplace=True)
odoo_locations_temp_parent_dataframe.rename(columns={'temporary_parent_id': 'location_id/id'}, inplace=True)
odoo_locations_temp_parent_dataframe.to_csv(f'{GENERATED_CSV_FILES_PATH}i5.locations.temp.parent.csv', index=False)

# Create import file with the locations with actual parent
odoo_locations_parents_dataframe = transformation_result_dataframe[['id', 'location_id/id']]
odoo_locations_parents_dataframe.to_csv(f'{GENERATED_CSV_FILES_PATH}i5.locations.parents.csv', index=False)


**Load i5 locations into Odoo**

In [ ]:
model_name = 'stock.location'
# Load the i5 locations files with temporary parent
import_data(file_csv='i5.locations.temp.parent.csv', model_name=model_name)
import_data(file_csv='i5.locations.parents.csv', model_name=model_name)

**Transform Product Categories**

In [ ]:
model_name = 'product.category'

i5_categories_dataframe = pandas.read_csv(f'{INPUT_CSV_FILES_PATH}Group Descriptions.tsv', sep='\t')

i5_categories_dataframe = i5_categories_dataframe.fillna('')
col_names = ['GROUP', 'GRDESC', 'STYLE', 'DESC']
for col_name in col_names:
    i5_categories_dataframe[col_name] = i5_categories_dataframe[col_name].astype(str)
    i5_categories_dataframe[col_name] = i5_categories_dataframe[col_name].str.strip()

odoo_parents_categories_dataframe= pandas.DataFrame(columns=['id', 'name', 'parent_id/id', 'group'])

def build_category_id(category_name):
    return f'i5_migration.product.category.{category_name.lower().replace(" ", "_").replace("/", "_")}'

ids_already_created = set()
for row in i5_categories_dataframe.iterrows():
    category_name = row[1].GRDESC
    category_id = build_category_id(category_name)
    parent_category_id = 'product.product_category_all'

    if category_id not in ids_already_created:
        ids_already_created.add(category_id)
        odoo_parents_categories_dataframe = odoo_parents_categories_dataframe.append({
            'id': category_id,
            'name': category_name,
            'parent_id/id': parent_category_id,
            'group': row[1].GROUP
        }, ignore_index=True)

odoo_parents_categories_dataframe.to_csv(f'{GENERATED_CSV_FILES_PATH}i5.categories.parents.csv', index=False)

odoo_child_categories_dataframe = pandas.DataFrame(columns=['id', 'name', 'parent_id/id'])

def build_child_category_id(category_name, style_code):
    return f'i5_migration.product.category.{category_name.lower().replace(" ", "_").replace("/", "_")}_child_{style_code.lower()}'

for row in i5_categories_dataframe.iterrows():
    category_name = row[1].DESC
    style_code = row[1].STYLE
    category_id = build_child_category_id(row[1].GRDESC, style_code)
    parent_id = build_category_id(row[1].GRDESC)
    odoo_child_categories_dataframe = odoo_child_categories_dataframe.append({
        'id': category_id,
        'name': category_name,
        'parent_id/id': parent_id,
        'group': row[1].GROUP,
        'style': style_code,
    }, ignore_index=True)

odoo_child_categories_dataframe.to_csv(f'{GENERATED_CSV_FILES_PATH}i5.categories.children.csv', index=False)


**Load product.categories**

In [ ]:
model_name = 'product.category'

import_data(file_csv='i5.categories.parents.csv', model_name=model_name, ignore_fields=['group', 'style'])
import_data(file_csv='i5.categories.children.csv', model_name=model_name, ignore_fields=['group', 'style'])

**Transform Products**

In [ ]:
model_name = 'product.template'

categories_dataframe = pandas.read_csv(f'{GENERATED_CSV_FILES_PATH}i5.categories.children.csv')[['id', 'group', 'style']]
categories_dataframe.rename(columns={'id': 'categ_id/id'}, inplace=True)
col_names = ['categ_id/id', 'group', 'style']
for col_name in col_names:
    categories_dataframe[col_name] = categories_dataframe[col_name].astype(str)
    categories_dataframe[col_name] = categories_dataframe[col_name].str.strip()
odoo_products_dataframe = pandas.read_csv(f'{GENERATED_CSV_FILES_PATH}i5.products.csv')

i5_products_dataframe = pandas.read_csv(f'{INPUT_CSV_FILES_PATH}MasterProductsList.csv')

col_names = ['IMDESC', 'ITMDSC', 'PRICCD', 'IMLIST', 'IMTCST', 'GROUP', 'IMSTYL', 'IMRANK', 'IMGEN', 'IMXRBC', 'vendor_code', 'AMNAME', 'ITEM', 'IMMCST']
for col_name in col_names:
    i5_products_dataframe[col_name] = i5_products_dataframe[col_name].astype(str)
    i5_products_dataframe[col_name] = i5_products_dataframe[col_name].str.strip()

def get_purchase_uom(row):
    if row.PRICCD == '0':
        return 'uom.product_uom_unit'
    elif row.PRICCD == '1':
        return '__export__.product_uom_21_fd9caca2'
    elif row.PRICCD == '2':
        return '__export__.product_uom_20_fff5877f'
    else:
        return ''

odoo_products_dataframe= pandas.DataFrame(columns=['id', 'name', 'default_code', 'barcode', 'list_price', 'standard_price', 'uom_po_id', 'group', 'style'])
for row in i5_products_dataframe.itertuples():
    prod_name = row.IMDESC
    default_code = row.ITEM
    odoo_products_dataframe = odoo_products_dataframe.append({
      'id': f'i5_migration.product_template_{row.IMGEN}',
      'name': row.IMDESC + ' ' + row.ITMDSC,
      'default_code': row.IMGEN,
      'barcode': row.IMXRBC,
      'list_price': row.IMLIST,
      'standard_price': row.IMTCST,
      'type': 'product',
      'uom_po_id/id': get_purchase_uom(row),
      'group': row.GROUP,
      'style': row.IMSTYL,
      'vendor_account_number': row.vendor_code,
      'price': row.IMMCST, # Vendors price,
      'product_code': row.ITEM,
    }, ignore_index=True)

odoo_products_dataframe.to_csv(f'{GENERATED_CSV_FILES_PATH}i5.products.csv', index=False)

odoo_products_dataframe = odoo_products_dataframe.merge(categories_dataframe, how='left', on=['group', 'style'])
# replace NaN values with undefined category
odoo_products_dataframe[['categ_id/id']] = odoo_products_dataframe[['categ_id/id']].fillna(value='i5_migration.product.category.national_accounts_child_z7')
odoo_products_dataframe.drop(columns=['group', 'style'], inplace=True)
odoo_products_dataframe.to_csv(f'{GENERATED_CSV_FILES_PATH}i5.products2.csv', index=False)

## Service Items ##
i5_service_items_dataframe = pandas.read_csv(f'{INPUT_CSV_FILES_PATH}service_items.tsv', sep='\t')

col_names = ['IMDESC', 'ITMDSC', 'PRICCD', 'IMLIST', 'IMTCST', 'GROUP', 'IMSTYL', 'IMRANK', 'IMGEN', 'IMXRBC', 'AMNAME', 'ITEM', 'IMMCST']
for col_name in col_names:
    i5_service_items_dataframe[col_name] = i5_service_items_dataframe[col_name].astype(str)
    i5_service_items_dataframe[col_name] = i5_service_items_dataframe[col_name].str.strip()
i5_service_items_dataframe['GROUP'] = i5_service_items_dataframe['GROUP'].str.replace('.0', '')

odoo_service_items_dataframe= pandas.DataFrame(columns=['id', 'name', 'default_code', 'barcode', 'list_price', 'standard_price', 'uom_po_id', 'group', 'style'])
for row in i5_service_items_dataframe.itertuples():
    prod_name = row.IMDESC
    default_code = row.ITEM
    odoo_service_items_dataframe = odoo_service_items_dataframe.append({
      'id': f'i5_migration.product_template_{row.IMGEN}',
      'name': row.IMDESC + ' ' + row.ITMDSC,
      'default_code': row.IMGEN,
      'barcode': row.IMXRBC,
      'list_price': row.IMLIST,
      'standard_price': row.IMTCST,
      'type': 'service',
      'uom_po_id/id': get_purchase_uom(row),
      'group': str(int(row.GROUP)) if row.GROUP != 'nan' else '',
      'style': row.IMSTYL,
      'vendor_name': row.AMNAME,
      'price': row.IMMCST, # Vendors price,
      'product_code': row.ITEM,
    }, ignore_index=True)
odoo_service_items_dataframe.to_csv(f'{GENERATED_CSV_FILES_PATH}i5.products.service.items.csv', index=False)

odoo_service_items_dataframe = odoo_service_items_dataframe.merge(categories_dataframe, how='left', on=['group', 'style'])
# replace NaN values with undefined category
odoo_service_items_dataframe[['categ_id/id']] = odoo_service_items_dataframe[['categ_id/id']].fillna(value='i5_migration.product.category.national_accounts_child_z7')
odoo_service_items_dataframe.drop(columns=['group', 'style'], inplace=True)


vendors_dataframe = pandas.read_csv(f'{GENERATED_CSV_FILES_PATH}i5.vendors.csv')[['id', 'name']]
vendors_dataframe.rename(columns={'name': 'vendor_name'}, inplace=True)
vendors_dataframe.rename(columns={'id': 'name/id'}, inplace=True)
odoo_service_items_dataframe.rename(columns={'AMNAME': 'vendor_name'}, inplace=True)

odoo_service_items_suppliers_dataframe = odoo_service_items_dataframe.merge(vendors_dataframe, how='inner', on=['vendor_name'])
# merge with vendors to get vendor id
odoo_service_items_dataframe = odoo_service_items_dataframe.merge(vendors_dataframe, how='left', on=['vendor_name'])

odoo_service_items_dataframe.to_csv(f'{GENERATED_CSV_FILES_PATH}i5.products.template.service.items.csv', index=False)
odoo_service_items_suppliers_dataframe.to_csv(f'{GENERATED_CSV_FILES_PATH}i5.products.service.suppliers.csv', index=False)


**Load products**

In [ ]:
model_name = 'product.template'

import_data(model_name, file_csv='i5.products2.csv', context={'create_product_product': True, 'tracking_disable': True},
            ignore_fields=['vendor_account_number', 'price', 'product_code', 'uom_po_id', 'barcode'])
import_data(model_name, file_csv='i5.products.template.service.items.csv', context={'create_product_product': True, 'tracking_disable': True},
            ignore_fields=['vendor_name', 'price', 'product_code', 'name/id', 'uom_po_id', 'barcode'])

**Create product.product import file**

In [ ]:
model_name = 'product.product'

products_dataframe = pandas.read_csv(f'{GENERATED_CSV_FILES_PATH}i5.products2.csv')[['id', 'name', 'default_code']]
services_dataframe = pandas.read_csv(f'{GENERATED_CSV_FILES_PATH}i5.products.template.service.items.csv')[['id', 'name', 'default_code']]
products_dataframe = pandas.concat([products_dataframe, services_dataframe], ignore_index=True)

products_dataframe.rename(columns={'id': 'product_tmpl_id/id'}, inplace=True)
products_dataframe[['active']] = True
# create the id for the product.product using the default_code
products_dataframe['id'] = products_dataframe['default_code'].apply(lambda x: f'i5_migration.product.product_{x}')

products_dataframe.drop(columns=['default_code'], inplace=True)

products_dataframe.to_csv(f'{GENERATED_CSV_FILES_PATH}i5.product.product.csv', index=False)


**Load product.product**

In [ ]:
model_name = 'product.product'

import_data(model_name, file_csv='i5.product.product.csv')

**Transform products' supplier info**

In [ ]:
model_name = 'product.supplierinfo'

products_dataframe = pandas.read_csv(f'{GENERATED_CSV_FILES_PATH}i5.products2.csv')
products_dataframe = products_dataframe[['id', 'vendor_account_number', 'price', 'product_code', 'uom_po_id/id']]
products_dataframe.rename(columns={'id': 'product_tmpl_id/id', 'uom_po_id/id': 'product_uom/id'}, inplace=True)

vendors_dataframe = pandas.read_csv(f'{GENERATED_CSV_FILES_PATH}i5.vendors.csv')
vendors_dataframe = vendors_dataframe[['id', 'vendor_account_number']]
vendors_dataframe.rename(columns={'id': 'name/id'}, inplace=True)

odoo_products_suppliers_dataframe = products_dataframe.merge(vendors_dataframe, how='inner', on='vendor_account_number')
# Add columns with default values
odoo_products_suppliers_dataframe['delay'] = 0
odoo_products_suppliers_dataframe['min_qty'] = 1
# Generate id for each row with the format i5_migration.product.supplierinfo_{row_index}
odoo_products_suppliers_dataframe['id'] = odoo_products_suppliers_dataframe.index.map(lambda x: f'i5_migration.product.supplierinfo_{x}')
odoo_products_suppliers_dataframe.drop(columns=['vendor_account_number'], inplace=True)

odoo_products_suppliers_dataframe.to_csv(f'{GENERATED_CSV_FILES_PATH}i5.products.suppliers.csv', index=False)

## Service items ##
services_suppliers_dataframe = pandas.read_csv(f'{GENERATED_CSV_FILES_PATH}i5.products.service.suppliers.csv')
services_suppliers_dataframe = services_suppliers_dataframe[['id', 'name/id', 'price', 'product_code', 'uom_po_id/id']]
services_suppliers_dataframe.rename(columns={'id': 'product_tmpl_id/id', 'uom_po_id/id': 'product_uom/id'}, inplace=True)
services_suppliers_dataframe['id'] = services_suppliers_dataframe.index.map(lambda x: f'i5_migration.product.supplierinfo_service_{x}')
services_suppliers_dataframe.to_csv(f'{GENERATED_CSV_FILES_PATH}i5.products.suppliers.services.csv', index=False)


**Load products' supplier info**

In [ ]:
model_name = 'product.supplierinfo'

import_data(model_name, 'i5.products.suppliers.csv')
import_data(model_name, 'i5.products.suppliers.services.csv')
